# Tutorial 3: performing physical optics propagations.

In this third tutorial, we take the same optical system as in tutorial 2. However, instead of performing ray-traces through the system, we perform physical optics (PO).

In [ ]:
%matplotlib notebook

import numpy as np

from src.PyPO.System import System

s = System()

In [ ]:
plane_focus = {
            "name"      : "plane_focus",
            "gmode"     : "uv",
            "lims_u"    : np.array([0, 0.1]),
            "lims_v"    : np.array([0, 360]),
            "gridsize"  : np.array([101, 101])
            }

s.addPlane(plane_focus)

GPODict = {                                                                                                                                                                     
    "name"      : "focus",                                                                                                                                  
    "lam"       : 0.01,                                                                                                      
    "w0x"       : 0.05,                                                                                             
    "w0y"       : 0.05,                                                                                             
    "n"         : 1,                                                                                                                             
    "E0"        : 1,                                                                                                                                  
    "dxyz"      : 0,                                                                                                 
    "pol"       : np.array([1, 0, 0])                                                                                                          
}

s.createGaussian(GPODict, "plane_focus")
s.plotBeam2D("focus", "Ex", vmin=-30, vmax=0)

s.translateGrids("plane_focus", np.array([0, 0, 100]))


We start by defining a plane in the upper focus of the ellipsoid. We give it a radius of 0.1 mm. However, we do not place it in the focus just yet. First, we define a Gaussian PO beam on the plane. Because the Gaussian PO beams are always defined with their focus at x=y=z=0, we need to be careful to define this before we translate the focal plane.

The Gaussian beam itself is created by passing a filled dictionary, called GPODict here, to the s.createGaussian methos. The dictionary consists of several fields. The 'name' field contains the name by which the field will be stored in the system. 'lam' sets the wavelength, 0.01 mm in this case. After that, 'w0x' and 'w0y' set the beamwaist sizes along the x and y-axis in mm, respectively. The refractive index of the medium in which the beam is defined is set by 'n'. The peak electric field value at the focus is set by 'E0'. The astigmatic z-axis distance between the focus of the beam in the x-plane and the y-plane in mm is set by 'dxyz'. Note that, if this option is used, the focus in the x-plane is placed at z=0 and the focus in the y-plane at z=-dxyz. The final field, 'pol', sets the polarisation vector of the Gaussian beam.

In [ ]:
ellipse = {
            "name"      : "ellipsoid",
            "pmode"     : "focus",
            "gmode"     : "uv",
            "flip"      : True,
            "focus_1"   : np.array([0, 0, 100]),
            "focus_2"   : np.array([0, 0, -100]),
            "orient"    : "z",
            "ecc"       : 0.5,
            "lims_u"    : np.array([0, 10]),
            "lims_v"    : np.array([0, 360]),
            "gridsize"  : np.array([301, 301])
            }

s.addEllipse(ellipse)

plane_t = {
            "name"      : "plane_t",
            "gmode"     : "uv",
            "lims_u"    : np.array([0, 5]),
            "lims_v"    : np.array([0, 360]),
            "gridsize"  : np.array([301, 301])
            }

s.addPlane(plane_t)
s.rotateGrids("plane_t", np.array([45, 0, 0]))

parabola = {
            "name"      : "paraboloid",
            "pmode"     : "focus",
            "gmode"     : "uv",
            "focus_1"   : np.array([0, -100, 0]),
            "vertex"    : np.array([-10, -100, 0]),
            "lims_u"    : np.array([0, 0.5]),
            "lims_v"    : np.array([0, 360]),
            "gcenter"   : np.array([0,-20]),
            "gridsize"  : np.array([301, 301])
            }

s.addParabola(parabola)

plane_out = {
            "name"      : "plane_out",
            "gmode"     : "uv",
            "lims_u"    : np.array([0, 0.5]),
            "lims_v"    : np.array([0, 360]),
            "gridsize"  : np.array([301, 301])
            }

s.addPlane(plane_out)
s.rotateGrids("plane_out", np.array([0, -90, 0]))
s.translateGrids("plane_out", np.array([50, -120, 0]))

Here, we basically re-create the system from the previous tutorial. However, as we are doing PO now, the size of the reflectors become important. In the previous tutorial we purposefully oversized the off-axis paraboloid reflector for illustrative purposes. This time, we size the paraboloid in such a way that the illuminating beam has an edge taper between -10 and -15 dB.

In [ ]:
focus_to_ell_PO = {
    "t_name"      : "ellipsoid",
    "s_current"   : "focus",
    "mode"        : "JMEH",
    "name_JM"     : "JM_ell",
    "name_EH"     : "EH_ell",
    "epsilon"     : 10
}

s.runPO(focus_to_ell_PO)
#s.plotBeam2D("EH_ell", "Ex", project="xy", vmin=-30, vmax=0)

ell_to_plane_t_PO = {
    "t_name"      : "plane_t",
    "s_current"   : "JM_ell",
    "mode"        : "JMEH",
    "name_JM"     : "JM_pt",
    "name_EH"     : "EH_pt",
    "epsilon"     : 10
}

s.runPO(ell_to_plane_t_PO)
#s.plotBeam2D("EH_pt", "Ex", project="xy", vmin=-30, vmax=0)

plane_t_to_par_PO = {
    "t_name"      : "paraboloid",
    "s_current"   : "JM_pt",
    "mode"        : "JMEH",
    "name_JM"     : "JM_par",
    "name_EH"     : "EH_par",
    "epsilon"     : 10
}

s.runPO(plane_t_to_par_PO)

In [ ]:
s.plotBeam2D("EH_par", "Ex", project="yz", vmin=-30, vmax=0)

In [ ]:
plane_ff = {
            "name"      : "plane_ff",
            "gmode"     : "AoE",
            "lims_Az"    : np.array([-0.7, 0.7]) * 6,
            "lims_El"    : np.array([-0.7, 0.7]) * 6 - 90,
            "gridsize"  : np.array([301, 301])
            }
s.addPlane(plane_ff)

par_to_plane_ff_PO = {
    "t_name"      : "plane_ff",
    "s_current"   : "JM_par",
    "mode"        : "FF",
    "name_EH"     : "EH_FF",
    "nThreads"    : 256,
    "device"      : "GPU",
    "epsilon"     : 10
}



s.runPO(par_to_plane_ff_PO)

s.plotBeam2D("EH_FF", "Ex", vmin=-30, vmax=0)
s.plotBeamCut("EH_FF", "Ex", vmin=-50, vmax=0)
hpbw_e, hpbw_h = s.calcHPBW("EH_FF", "Ex")
print(s.calcMainBeam("EH_FF", "Ex"))
print(hpbw_e/3600)
print(hpbw_h/3600)

In [ ]:
mb1_nom = s.calcMainBeam("EH_FF", "Ex")
Ap = np.pi * 0.5**2
Omega_mb1 = np.pi * np.radians(hpbw_e/3600) * np.radians(hpbw_h/3600) / (4 * np.log(2))
eta_ap_mb1 = (0.01**2 / Ap) * mb1_nom / Omega_mb1
print(eta_ap_mb1)